In [1]:
import numpy as np
import pandas as pd
import os
import re
from pathlib import Path
import difflib
from collections import Counter
#from metaphone import doublemetaphone
import recordlinkage
from recordlinkage.preprocessing import clean

ROOT = '/home/thabib/database_records_linkage'

In [2]:
s1_cstr = pd.read_csv(os.path.join(ROOT, 'data', 'source1_cstr.csv'))
s2_cstr_parsed = pd.read_csv(os.path.join(ROOT, 'data', 'source2_cstr_parsedaddress.csv'))

/tmp/ipykernel_33527/671281498.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  s2_cstr_parsed = pd.read_csv(os.path.join(ROOT, 'data', 'source2_cstr_parsedaddress.csv'))


In [3]:
display(s1_cstr)
display(s2_cstr_parsed)

,id,name,street_number,street_type,street_name,address_line2,postal_code,city
0,542012000015,societe des sucreries du marquenterre,NaN,rue,de la fontaine,NaN,80120,rue
1,542012000031,societe des sucreries du marquenterre,NaN,rte,nationale,NaN,62140,marconnelle
2,545011900028,le grand cafe et gassendi,NaN,NaN,saint christophe,route de marseille,4000,digne les bains
3,552017600016,hernas cartonnage,50.0,rue,pasteur,NaN,80210,feuquieres en vimeu
4,552017600032,hernas cartonnage,NaN,NaN,zone des hayettes,NaN,80210,chepy
...,...,...,...,...,...,...,...,...
1826164,90771119600010,bionormande,NaN,rue,de la grande flandre,lot 1 7 a,76270,neufchatel en bray
1826165,90771120400012,ahela bat,18.0,rue,saint clement,NaN,93200,saint denis
1826166,90771121200015,deco batiment,7.0,rue,auguste blanqui,NaN,93200,saint denis
1826167,90760906900014,chateaux merveilles,15.0,rue,de milan,NaN,75009,paris 9


,website,id,name,street_number,street_type,street_name,address_line2,postal_code,city
0,NaN,NaN,sandrine berny,226.0,rue,menthon,NaN,1380,saint cyr sur menthon
1,NaN,8.412263e+13,sarl landrix freres,50.0,rte,bage,NaN,1380,saint cyr sur menthon
2,NaN,7.913458e+13,marion kwasniewski,120.0,rue,menthon,NaN,1380,saint cyr sur menthon
3,NaN,NaN,claude fondation,28.0,chem,ruillat,moulin,1380,saint cyr sur menthon
4,NaN,4.341618e+13,manigand stephane,350.0,rte,loeze,NaN,1380,saint cyr sur menthon
...,...,...,...,...,...,...,...,...,...
3530297,NaN,8.497810e+13,jacoby o,NaN,NaN,NaN,NaN,NaN,NaN
3530298,http://agence.carrefourlocation.fr,NaN,carrefour location,NaN,NaN,quart guardere,NaN,83150,bandol
3530299,http://aquabike-en-mer.com,7.951850e+13,calvet richard,5.0,rue,jardins,NaN,83150,bandol
3530300,http://dolcefarnientebandol.com,5.392586e+13,dolce far niente,54.0,NaN,traverse laboureur,NaN,83150,bandol


# Using Recordlinkage

see here: https://recordlinkage.readthedocs.io/en/latest/about.html & https://pbpython.com/record-linking.html

### Matching rows using recordlinkage

In [5]:
indexer = recordlinkage.Index()
indexer.block(left_on=['city', 'name'], right_on=['city', 'name'])
candidates = indexer.index(s1_cstr, s2_cstr_parsed)
len(candidates)

336192

In [ ]:
threshold = 0.8
comp = recordlinkage.Compare(n_jobs=-1)
comp.string(left_on='name', right_on='name', threshold=threshold, label='name', missing_value=-1, method='cosine')
comp.string(left_on='street_name', right_on='street_name', threshold=threshold, label='street_name', missing_value=-1, method='cosine')
comp.string(left_on='street_type', right_on='street_type', threshold=threshold, label='street_type', missing_value=-1, method='cosine')
comp.string(left_on='street_name', right_on='street_name', threshold=threshold, label='street_name', missing_value=-1, method='cosine')
comp.string(left_on='city', right_on='city', threshold=threshold, label='city', missing_value=-1, method='cosine')
comp.exact(left_on='street_number', right_on='street_number', label='street_number', missing_value=-1)
comp.exact(left_on='postal_code', right_on='postal_code', label='postal_code', missing_value=-1)
features = comp.compute(pairs=candidates, x=s1_cstr, x_link=s2_cstr_parsed)

In [11]:
indexer = recordlinkage.Index()
indexer.sortedneighbourhood(left_on=['name'], right_on=['name'], 
                            block_left_on=['city'], block_right_on=['city'])
candidates = indexer.index(x=s1_cstr, x_link=s2_cstr_parsed)
len(candidates)

421298

In [ ]:
threshold = 0.8
comp = recordlinkage.Compare(n_jobs=-1)
comp.string(left_on='name', right_on='name', threshold=threshold, label='name', missing_value=-1, method='cosine')
comp.string(left_on='street_name', right_on='street_name', threshold=threshold, label='street_name', missing_value=-1, method='cosine')
comp.string(left_on='street_type', right_on='street_type', threshold=threshold, label='street_type', missing_value=-1, method='cosine')
comp.string(left_on='street_name', right_on='street_name', threshold=threshold, label='street_name', missing_value=-1, method='cosine')
comp.string(left_on='city', right_on='city', threshold=threshold, label='city', missing_value=-1, method='cosine')
comp.exact(left_on='street_number', right_on='street_number', label='street_number', missing_value=-1)
comp.exact(left_on='postal_code', right_on='postal_code', label='postal_code', missing_value=-1)
features = comp.compute(pairs=candidates, x=s1_cstr, x_link=s2_cstr_parsed)

### Parsing 'address' in source2 into separate columns

In [5]:
from postal.parser import parse_address

# 'city' – city
# 'postcode' – psotal_code
# 'house_number' – street_number
# 'road' – street_name
# 'house' OR anything else – address_line2

In [6]:
unique_street_types = np.unique(s1_cstr['street_type'].dropna())
unique_street_types

array(['all', 'av', 'bd', 'cami', 'car', 'che', 'chem', 'chs', 'cite',
       'clos', 'cor', 'cote', 'cour', 'crs', 'dom', 'dsc', 'eca', 'esp',
       'fg', 'gr', 'ham', 'hle', 'imp', 'ld', 'lot', 'mar', 'mte', 'parc',
       'pas', 'pl', 'plan', 'pln', 'plt', 'pont', 'port', 'pro', 'prv',
       'qua', 'quai', 'res', 'rle', 'roc', 'rpt', 'rte', 'rue', 'sen',
       'sq', 'tpl', 'tra', 'vla', 'vlge', 'voie', 'za', 'zac', 'zi',
       'zone'], dtype=object)

In [85]:
parse_address('22 Bis r Nationale, 45190 BEAUGENCY')

[('22 bis', 'house_number'),
 ('r nationale', 'road'),
 ('45190', 'postcode'),
 ('beaugency', 'city')]

In [87]:
parse_address('Parking Station De Lavage 505 av Georges Pompidou, 73200 GILLY SUR ISÈRE')

[('parking station de lavage', 'house'),
 ('505', 'house_number'),
 ('av georges pompidou', 'road'),
 ('73200', 'postcode'),
 ('gilly sur isère', 'city')]

In [4]:
parse_address('4 BD DE LA REPUBLIQUE,80100 ABBEVILLE')

[('4', 'house_number'),
 ('bd de la republique', 'road'),
 ('80100', 'postcode'),
 ('abbeville', 'city')]

In [5]:
parse_address('Zac De Cadréan bd Cadréan, 44550 MONTOIR DE BRETAGNE')

[('zac de cadréan', 'house'),
 ('bd cadréan', 'road'),
 ('44550', 'postcode'),
 ('montoir de bretagne', 'city')]

In [32]:
parse_address('zi Louis Delaporte, 76370 ROUXMESNIL BOUTEILLES')

[('zi louis delaporte', 'road'),
 ('76370', 'postcode'),
 ('rouxmesnil bouteilles', 'city')]

In [7]:
def parse_address_fields(address=None):
    
    dict_address = {
        'street_number': np.nan,
        'street_type': np.nan,
        'street_name': np.nan,
        'address_line2': np.nan,
        'postal_code': np.nan,
        'city': np.nan
    }
    
    if type(address) == float:
        pass
    
    else:
        addline2 = ''
        parsed_add = parse_address(address)
        
        for item in parsed_add:
            
            if item[1] == 'city':
                dict_address['city'] = item[0]
                
            elif item[1] == 'postcode':
                try:
                    dict_address['postal_code'] = int(item[0])
                except ValueError:
                    dict_address['postal_code'] = item[0]
                    
            elif item[1] == 'road':
                road_comps = item[0].split()
                dict_address['street_name'] = ' '.join(road_comps[1:])
                if road_comps[0] == 'r':
                    dict_address['street_type'] = 'rue'
                elif road_comps[0] in unique_street_types:
                    dict_address['street_type'] = road_comps[0]
                else:
                    dict_address['street_name'] = item[0]
                    
            elif item[1] == 'house_number':
                try:
                    dict_address['street_number'] = int(item[0])
                except ValueError:
                    #print('The parsed \'house_number\' is not a valid integer. Extracting the first integer '\
                     #     'occurrence (from string) and setting it as \'street_number\'')
                    ints = re.findall(r'\d+', item[0])
                    if len(ints) == 0:
                        dict_address['street_number'] = np.nan
                    else:
                        sep_ints = [int(s) for s in item[0].split() if s.isdigit()]
                        if len(sep_ints) != 0:
                            dict_address['street_number'] = sep_ints[0]
                        else:
                            dict_address['street_number'] = ints[0]
                            
            else:
                addline2 += item[0]
                addline2 += ' '
        if addline2 == '':
            dict_address['address_line2'] = np.nan
        else:
            dict_address['address_line2'] = addline2

    return dict_address

In [8]:
s2_cstr[['street_number','street_type','street_name','address_line2', 'postal_code','city']] = s2_cstr.apply(lambda row: parse_address_fields(row.address), axis=1, result_type='expand')

In [9]:
s2_cstr

,address,website,id,name,street_number,street_type,street_name,address_line2,postal_code,city
0,226 r menthon 01380 saint cyr sur menthon,NaN,NaN,sandrine berny,226,rue,menthon,NaN,1380,saint cyr sur menthon
1,50 rte bage 01380 saint cyr sur menthon,NaN,8.412263e+13,sarl landrix freres,50,rte,bage,NaN,1380,saint cyr sur menthon
2,120 r menthon 01380 saint cyr sur menthon,NaN,7.913458e+13,marion kwasniewski,120,rue,menthon,NaN,1380,saint cyr sur menthon
3,moulin 28 chem ruillat 01380 saint cyr sur men...,NaN,NaN,claude fondation,28,chem,ruillat,moulin,1380,saint cyr sur menthon
4,350 rte loeze 01380 saint cyr sur menthon,NaN,4.341618e+13,manigand stephane,350,rte,loeze,NaN,1380,saint cyr sur menthon
...,...,...,...,...,...,...,...,...,...,...
3530297,NaN,NaN,8.497810e+13,jacoby o,NaN,NaN,NaN,NaN,NaN,NaN
3530298,quart guardere 83150 bandol,http://agence.carrefourlocation.fr,NaN,carrefour location,NaN,NaN,quart guardere,NaN,83150,bandol
3530299,5 r jardins 83150 bandol,http://aquabike-en-mer.com,7.951850e+13,calvet richard,5,rue,jardins,NaN,83150,bandol
3530300,54 traverse laboureur 83150 bandol,http://dolcefarnientebandol.com,5.392586e+13,dolce far niente,54,NaN,traverse laboureur,NaN,83150,bandol


In [10]:
s2_cstr_parsed_address = s2_cstr.drop('address', axis=1)

In [12]:
s2_cstr_parsed_address.to_csv(os.path.join(ROOT, 'data', 'source2_cstr_parsedaddress.csv'), index=False)